In [ ]:
from gs_quant.session import GsSession

GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
# create what if trade

from gs_quant.instrument import FXOption, FXEuropeanKnockout
from gs_quant.markets.portfolio import Portfolio, Grid
from gs_quant.markets import PricingContext, LiveMarket
from gs_quant.risk import FXSpot, FXAnnualImpliedVol
import datetime as dt
import pprint

pp = pprint.PrettyPrinter(indent=4)

# shift+tab to preview what the parameters should be
leg = FXOption(
    pair='EURUSD',
    expiration_date='3m',
    strike_price='25D',
    notional_amount='50m',
    premium=0,
    option_type='Call',
    buy_sell='Buy',
)
pp.pprint(leg.as_dict())

leg.price()

# by default the pricing is done as of today with last nights close rolled to today

In [ ]:
# if we wanted to price a trade historically in time we can do that using an historical pricing context

from gs_quant.markets import HistoricalPricingContext

with HistoricalPricingContext(
    start=dt.date(2025, 1, 3), end=dt.date(2025, 4, 1), visible_to_gs=True, market_data_location='NYC'
):
    h = leg.price()
h.result().plot()

In [ ]:
# when we price an Spot+1% trade historically what does that mean.  Well that depends on whether you have resolved the trade
# or not.
# If not then the trade will be moving in time, on each day a new 3m spot+1% option was priced.  We may want to fix the
# trade as at the start date

with PricingContext(pricing_date=dt.date(2025, 1, 3)):
    leg2 = leg.resolve(in_place=False)
leg2 = leg2.result()

# compare the unresolved trade to the resolved one.
pp.pprint(leg2.as_dict())


from gs_quant.risk import Price

with HistoricalPricingContext(start=dt.date(2025, 1, 3), end=dt.date(2025, 4, 1)):
    h2 = leg2.calc(Price)
h2.result().plot()

In [ ]:
#  We can price live by amending the pricing context as well

with PricingContext(market=LiveMarket()):
    s = leg.calc(FXSpot)
s.result()

In [ ]:
# we have a concept of a portfolio which behaves similarly to a trade so you can pack many trades into one object and price that.  Also shows how other metrics can be referenced and called.

from gs_quant.common import InOut
from gs_quant.risk import FXQuotedDelta

port = Portfolio(
    [
        FXEuropeanKnockout(
            pair='GBPUSD',
            expiration_date='3m',
            strike_price='1.25',
            knock_in_or_out=InOut.Out,
            barrier_level=f'{1.10 + x / 100}',
            name=f'{x}',
        )
        for x in range(25, 35)
    ]
)


print(len(port))
r = port.calc((FXQuotedDelta, FXAnnualImpliedVol))
r.to_frame()

In [ ]:
# screeners are a popular usecase and there is a type of portfolio called a grid which can aid in construction of 2d grids of prices

# grids take an example trade and then two parameters and the possible values of each
grid = Grid(FXOption(pair='EURUSD', premium=0), 'strike_price', ['10d', '20d'], 'expiration_date', ['1m', '3m', '6m'])

r = grid.price()
r.to_frame()

In [ ]:
# we can define and apply scenarios to our pricing

from gs_quant.risk import MarketDataPattern, MarketDataShock, MarketDataShockType, MarketDataShockBasedScenario

parallel_shock = MarketDataShockBasedScenario(
    shocks={
        MarketDataPattern(mkt_type='FX Vol', mkt_asset='USD/EUR', mkt_class='ATM VOL'): MarketDataShock(
            MarketDataShockType.Absolute, 0.0001
        )
    }
)

with PricingContext(dt.date(2025, 2, 18)):
    orig_price = leg.price()
    with parallel_shock:
        res = leg.price()
print(f'original price: {orig_price.result():.4f}')
print(f'shock price: {res.result():.4f}')

In [ ]:
# as well as pricing historically we can project the market forward and price in the future.

from gs_quant.risk import RollFwd

carry_scenario = RollFwd(date=dt.date(2025, 3, 15), realise_fwd=True)
with carry_scenario:
    res = leg.price()

print(f'roll fwd price: {res:.4f}')

In [ ]:
# sometimes you may want to look at the market data used in a pricing request.

from gs_quant.markets import PricingContext

with PricingContext(dt.date(2024, 4, 4)):
    market = leg.market()
market.result().market_data_dict

In [ ]:
%%time
# loading from a dataset


from gs_quant.data import Dataset
import datetime as dt


dataset = Dataset('MACRO_EVENTS_CALENDAR')
dataset.get_data(dt.datetime(2024, 12, 1), dt.datetime(2025, 12, 31), eventName='ISM Manufacturing PMI')

In [ ]:
irfwd_eur = Dataset('IR_SWAP_RATES').get_data(
    dt.date(2025, 1, 3),
    dt.date(2025, 1, 29),
    currency='EUR',
    effectiveTenor='1y',
    terminationTenor='10y',
    floatingRateOption='EUR-EURIBOR-TELERATE',
    floatingRateDesignatedMaturity='6m',
    clearingHouse='LCH',
)
irfwd_eur['rate']